# General Dataset Utilities

Authors: [Tom Vo](https://github.com/tomvothecoder/) & [Stephen Po-Chedley](https://github.com/pochedls/)

Updated: 11/07/24 [xcdat v0.7.3]


## Overview

This notebook demonstrates the use of general utility methods available in `xcdat`, including
the reorientation of the longitude axis, centering of time coordinates using time bounds, and
adding and getting bounds.

The data used in this example can be found in the [xarray-data repository](https://github.com/pydata/xarray-data).


### Notebook Kernel Setup

Users can [install their own instance of xcdat](../getting-started-guide/installation.rst) and follow these examples using their own environment (e.g., with VS Code, Jupyter, Spyder, iPython) or [enable xcdat with existing JupyterHub instances](../getting-started-guide/getting-started-hpc-jupyter.rst).

First, create the conda environment:

```bash
conda create -n xcdat_notebook -c conda-forge xcdat xesmf matplotlib ipython ipykernel cartopy nc-time-axis gsw-xarray jupyter pooch
```

Then install the kernel from the `xcdat_notebook` environment using `ipykernel` and name the kernel with the display name (e.g., `xcdat_notebook`):

```bash
python -m ipykernel install --user --name xcdat_notebook --display-name xcdat_notebook
```

Then to select the kernel `xcdat_notebook` in Jupyter to use this kernel.


In [1]:
import xcdat as xc

## Open a dataset

Datasets can be opened and read using `open_dataset()` or `open_mfdataset()` (multi-file).

Related APIs: [xcdat.open_dataset()](../generated/xcdat.open_dataset.rst) & [xcdat.open_mfdataset()](../generated/xcdat.open_mfdataset.rst)


Let's use the example dataset and save it to netCDF (`.nc`), then open it with xCDAT.


In [4]:
ds = xc.tutorial.open_dataset("air_temperature", use_cftime=True)
ds.to_netcdf("air_temperature.nc")

/global/homes/v/vo13/mambaforge/envs/xcdat_notebook_0.7.3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: SerializationWarning: saving variable air with floating point data as an integer dtype without any _FillValue to use for NaNs
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
# NOTE: Opening a multi-file dataset will result in data variables to be dask
# arrays.
ds = xc.open_mfdataset("air_temperature.nc")
# print dataset
ds

<xarray.Dataset> Size: 31MB
Dimensions:    (lat: 25, time: 2920, lon: 53, bnds: 2)
Coordinates:
  * lat        (lat) float32 100B 75.0 72.5 70.0 67.5 ... 22.5 20.0 17.5 15.0
  * lon        (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time       (time) object 23kB 2013-01-01 00:00:00 ... 2014-12-31 18:00:00
Dimensions without coordinates: bnds
Data variables:
    air        (time, lat, lon) float64 31MB dask.array<chunksize=(2920, 25, 53), meta=np.ndarray>
    lon_bnds   (lon, bnds) float32 424B dask.array<chunksize=(53, 2), meta=np.ndarray>
    lat_bnds   (lat, bnds) float32 200B dask.array<chunksize=(25, 2), meta=np.ndarray>
    time_bnds  (time, bnds) object 47kB dask.array<chunksize=(2920, 2), meta=np.ndarray>
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

## Reorient the longitude axis

Longitude can be represented from 0 to 360 E or as 180 W to 180 E. `xcdat` allows you to convert between these axes systems.

- Related API: [xcdat.swap_lon_axis()](../generated/xcdat.swap_lon_axis.rst)
- Alternative solution: `xcdat.open_mfdataset(dataset_links, lon_orient=(-180, 180))`


In [6]:
ds.lon

<xarray.DataArray 'lon' (lon: 53)> Size: 212B
array([200. , 202.5, 205. , 207.5, 210. , 212.5, 215. , 217.5, 220. , 222.5,
       225. , 227.5, 230. , 232.5, 235. , 237.5, 240. , 242.5, 245. , 247.5,
       250. , 252.5, 255. , 257.5, 260. , 262.5, 265. , 267.5, 270. , 272.5,
       275. , 277.5, 280. , 282.5, 285. , 287.5, 290. , 292.5, 295. , 297.5,
       300. , 302.5, 305. , 307.5, 310. , 312.5, 315. , 317.5, 320. , 322.5,
       325. , 327.5, 330. ], dtype=float32)
Coordinates:
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
Attributes:
    standard_name:  longitude
    long_name:      Longitude
    units:          degrees_east
    axis:           X
    bounds:         lon_bnds

In [7]:
ds2 = xc.swap_lon_axis(ds, to=(-180, 180))

In [8]:
ds2.lon

<xarray.DataArray 'lon' (lon: 53)> Size: 212B
array([-160. , -157.5, -155. , -152.5, -150. , -147.5, -145. , -142.5, -140. ,
       -137.5, -135. , -132.5, -130. , -127.5, -125. , -122.5, -120. , -117.5,
       -115. , -112.5, -110. , -107.5, -105. , -102.5, -100. ,  -97.5,  -95. ,
        -92.5,  -90. ,  -87.5,  -85. ,  -82.5,  -80. ,  -77.5,  -75. ,  -72.5,
        -70. ,  -67.5,  -65. ,  -62.5,  -60. ,  -57.5,  -55. ,  -52.5,  -50. ,
        -47.5,  -45. ,  -42.5,  -40. ,  -37.5,  -35. ,  -32.5,  -30. ],
      dtype=float32)
Coordinates:
  * lon      (lon) float32 212B -160.0 -157.5 -155.0 ... -35.0 -32.5 -30.0
Attributes:
    standard_name:  longitude
    long_name:      Longitude
    units:          degrees_east
    axis:           X
    bounds:         lon_bnds

## Center the time coordinates

A given point of time often represents some time period (e.g., a monthly average). In this situation, data providers sometimes record the time as the beginning, middle, or end of the period. `center_times()` places the time coordinate in the center of the time interval (using time bounds to determine the center of the period).

- Related API: [xcdat.center_times()](../generated/xcdat.center_times.rst)
- Alternative solution: `xcdat.open_mfdataset(dataset_links, center_times=True)`

The time bounds used for centering time coordinates:


In [9]:
# We access the values with .values because it is a dask array.
ds.time_bnds.values

array([[cftime.DatetimeGregorian(2013, 1, 1, 0, 0, 0, 0, has_year_zero=False),
        cftime.DatetimeGregorian(2013, 1, 1, 6, 0, 0, 0, has_year_zero=False)],
       [cftime.DatetimeGregorian(2013, 1, 1, 6, 0, 0, 0, has_year_zero=False),
        cftime.DatetimeGregorian(2013, 1, 1, 12, 0, 0, 0, has_year_zero=False)],
       [cftime.DatetimeGregorian(2013, 1, 1, 12, 0, 0, 0, has_year_zero=False),
        cftime.DatetimeGregorian(2013, 1, 1, 18, 0, 0, 0, has_year_zero=False)],
       ...,
       [cftime.DatetimeGregorian(2014, 12, 31, 6, 0, 0, 0, has_year_zero=False),
        cftime.DatetimeGregorian(2014, 12, 31, 12, 0, 0, 0, has_year_zero=False)],
       [cftime.DatetimeGregorian(2014, 12, 31, 12, 0, 0, 0, has_year_zero=False),
        cftime.DatetimeGregorian(2014, 12, 31, 18, 0, 0, 0, has_year_zero=False)],
       [cftime.DatetimeGregorian(2014, 12, 31, 18, 0, 0, 0, has_year_zero=False),
        cftime.DatetimeGregorian(2015, 1, 1, 0, 0, 0, 0, has_year_zero=False)]],
      dtype=obje

Before centering time coordinates:


In [10]:
ds.time

<xarray.DataArray 'time' (time: 2920)> Size: 23kB
array([cftime.DatetimeGregorian(2013, 1, 1, 0, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(2013, 1, 1, 6, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(2013, 1, 1, 12, 0, 0, 0, has_year_zero=False),
       ...,
       cftime.DatetimeGregorian(2014, 12, 31, 6, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(2014, 12, 31, 12, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(2014, 12, 31, 18, 0, 0, 0, has_year_zero=False)],
      dtype=object)
Coordinates:
  * time     (time) object 23kB 2013-01-01 00:00:00 ... 2014-12-31 18:00:00
Attributes:
    standard_name:  time
    long_name:      Time
    bounds:         time_bnds

Now center the time coordinates:


In [11]:
ds3 = xc.center_times(ds)

After centering time coordinates:


In [12]:
ds3.time

<xarray.DataArray 'time' (time: 2920)> Size: 23kB
array([cftime.DatetimeGregorian(2013, 1, 1, 3, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(2013, 1, 1, 9, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(2013, 1, 1, 15, 0, 0, 0, has_year_zero=False),
       ...,
       cftime.DatetimeGregorian(2014, 12, 31, 9, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(2014, 12, 31, 15, 0, 0, 0, has_year_zero=False),
       cftime.DatetimeGregorian(2014, 12, 31, 21, 0, 0, 0, has_year_zero=False)],
      dtype=object)
Coordinates:
  * time     (time) object 23kB 2013-01-01 03:00:00 ... 2014-12-31 21:00:00
Attributes:
    standard_name:  time
    long_name:      Time
    bounds:         time_bnds

## Add bounds

Bounds are critical to many `xcdat` operations. For example, they are used in determining the weights in spatial or temporal averages and in regridding operations. `add_bounds()` will attempt to produce bounds if they do not exist in the original dataset.

- Related API: [xarray.Dataset.bounds.add_bounds()](../generated/xarray.Dataset.bounds.add_bounds.rst)
- Alternative solution: `xcdat.open_mfdataset(dataset_links, add_bounds=["X", "Y", "T"])`
  - (Assuming the file doesn't already have bounds for your desired axis/axes)


In [13]:
# We are dropping the existing bounds to demonstrate adding bounds.
# we are starting with the dataset with centered time points
ds4 = ds3.drop_vars("time_bnds")

In [14]:
try:
    ds4.bounds.get_bounds("T")
except KeyError as e:
    print(e)

"No bounds data variables were found for the 'T' axis. Make sure the dataset has bound data vars and their names match the 'bounds' attributes found on their related time coordinate variables. Alternatively, you can add bounds with `ds.bounds.add_missing_bounds()` or `ds.bounds.add_bounds()`."


There are two options for adding time bounds. The midpoint method places bounds at the midpoints between time bounds and the frequency method creates bounds based on the time stamp of each time point
and the frequency of the data. This is the midpoint method:


In [15]:
# midpoint method
ds4 = ds4.bounds.add_time_bounds(method="midpoint")
# print results
ds4.bounds.get_bounds("T")

<xarray.DataArray 'time_bnds' (time: 2920, bnds: 2)> Size: 47kB
array([[cftime.DatetimeGregorian(2013, 1, 1, 0, 0, 0, 0, has_year_zero=False),
        cftime.DatetimeGregorian(2013, 1, 1, 6, 0, 0, 0, has_year_zero=False)],
       [cftime.DatetimeGregorian(2013, 1, 1, 6, 0, 0, 0, has_year_zero=False),
        cftime.DatetimeGregorian(2013, 1, 1, 12, 0, 0, 0, has_year_zero=False)],
       [cftime.DatetimeGregorian(2013, 1, 1, 12, 0, 0, 0, has_year_zero=False),
        cftime.DatetimeGregorian(2013, 1, 1, 18, 0, 0, 0, has_year_zero=False)],
       ...,
       [cftime.DatetimeGregorian(2014, 12, 31, 6, 0, 0, 0, has_year_zero=False),
        cftime.DatetimeGregorian(2014, 12, 31, 12, 0, 0, 0, has_year_zero=False)],
       [cftime.DatetimeGregorian(2014, 12, 31, 12, 0, 0, 0, has_year_zero=False),
        cftime.DatetimeGregorian(2014, 12, 31, 18, 0, 0, 0, has_year_zero=False)],
       [cftime.DatetimeGregorian(2014, 12, 31, 18, 0, 0, 0, has_year_zero=False),
        cftime.DatetimeGregorian(2015, 1, 1, 0, 0, 0, 0, has_year_zero=False)]],
      dtype=object)
Coordinates:
  * time     (time) object 23kB 2013-01-01 03:00:00 ... 2014-12-31 21:00:00
Dimensions without coordinates: bnds
Attributes:
    xcdat_bounds:  True

Notice that the midpoint method does not place the bounds between the last moment of month `n` and the first moment of month `n+1`. The `frequency` method was meant to try to infer the correct bounds by taking into account the time stamps and the frequency of the data. The frequency method (below) is what is used when `add_bounds=["T"]` is specified in `open_dataset` or `open_mfdataset`.


In [16]:
# drop time bounds again
ds5 = ds4.drop_vars("time_bnds")
# timestamp / frequency method
ds5 = ds5.bounds.add_time_bounds(method="freq")
# print results
ds5.bounds.get_bounds("T")

<xarray.DataArray 'time_bnds' (time: 2920, bnds: 2)> Size: 47kB
array([[cftime.DatetimeGregorian(2013, 1, 1, 0, 0, 0, 0, has_year_zero=False),
        cftime.DatetimeGregorian(2013, 1, 1, 6, 0, 0, 0, has_year_zero=False)],
       [cftime.DatetimeGregorian(2013, 1, 1, 6, 0, 0, 0, has_year_zero=False),
        cftime.DatetimeGregorian(2013, 1, 1, 12, 0, 0, 0, has_year_zero=False)],
       [cftime.DatetimeGregorian(2013, 1, 1, 12, 0, 0, 0, has_year_zero=False),
        cftime.DatetimeGregorian(2013, 1, 1, 18, 0, 0, 0, has_year_zero=False)],
       ...,
       [cftime.DatetimeGregorian(2014, 12, 31, 6, 0, 0, 0, has_year_zero=False),
        cftime.DatetimeGregorian(2014, 12, 31, 12, 0, 0, 0, has_year_zero=False)],
       [cftime.DatetimeGregorian(2014, 12, 31, 12, 0, 0, 0, has_year_zero=False),
        cftime.DatetimeGregorian(2014, 12, 31, 18, 0, 0, 0, has_year_zero=False)],
       [cftime.DatetimeGregorian(2014, 12, 31, 18, 0, 0, 0, has_year_zero=False),
        cftime.DatetimeGregorian(2015, 1, 1, 0, 0, 0, 0, has_year_zero=False)]],
      dtype=object)
Coordinates:
  * time     (time) object 23kB 2013-01-01 03:00:00 ... 2014-12-31 21:00:00
Dimensions without coordinates: bnds
Attributes:
    xcdat_bounds:  True

Note that `ds.bounds.add_time_bounds(method="midpoint")` is the same as `ds.bounds.add_bounds("T")`. The latter method can be used to add bounds to other axes (e.g., latitude) as show below.


In [17]:
ds6 = ds.drop_vars("lat_bnds")
ds6 = ds6.bounds.add_bounds("Y")
ds6.lat_bnds

<xarray.DataArray 'lat_bnds' (lat: 25, bnds: 2)> Size: 200B
array([[76.25, 73.75],
       [73.75, 71.25],
       [71.25, 68.75],
       [68.75, 66.25],
       [66.25, 63.75],
       [63.75, 61.25],
       [61.25, 58.75],
       [58.75, 56.25],
       [56.25, 53.75],
       [53.75, 51.25],
       [51.25, 48.75],
       [48.75, 46.25],
       [46.25, 43.75],
       [43.75, 41.25],
       [41.25, 38.75],
       [38.75, 36.25],
       [36.25, 33.75],
       [33.75, 31.25],
       [31.25, 28.75],
       [28.75, 26.25],
       [26.25, 23.75],
       [23.75, 21.25],
       [21.25, 18.75],
       [18.75, 16.25],
       [16.25, 13.75]], dtype=float32)
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
Dimensions without coordinates: bnds
Attributes:
    xcdat_bounds:  True

## Add missing bounds for all axes supported by xcdat (X, Y, T, Z)

- Related API: [xarray.Dataset.bounds.add_missing_bounds()](../generated/xarray.Dataset.bounds.add_missing_bounds.rst)


In [18]:
# We drop the dataset axes bounds to demonstrate generating missing bounds.
ds7 = ds.drop_vars(["time_bnds", "lat_bnds", "lon_bnds"])

In [19]:
ds7

<xarray.Dataset> Size: 31MB
Dimensions:  (lat: 25, time: 2920, lon: 53)
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time     (time) object 23kB 2013-01-01 00:00:00 ... 2014-12-31 18:00:00
Data variables:
    air      (time, lat, lon) float64 31MB dask.array<chunksize=(2920, 25, 53), meta=np.ndarray>
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

In [20]:
# add now-missing bounds
ds7 = ds7.bounds.add_missing_bounds(["X", "Y", "T"])
# print dataset
ds7

<xarray.Dataset> Size: 31MB
Dimensions:    (lat: 25, time: 2920, lon: 53, bnds: 2)
Coordinates:
  * lat        (lat) float32 100B 75.0 72.5 70.0 67.5 ... 22.5 20.0 17.5 15.0
  * lon        (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time       (time) object 23kB 2013-01-01 00:00:00 ... 2014-12-31 18:00:00
Dimensions without coordinates: bnds
Data variables:
    air        (time, lat, lon) float64 31MB dask.array<chunksize=(2920, 25, 53), meta=np.ndarray>
    lon_bnds   (lon, bnds) float32 424B 198.8 201.2 201.2 ... 328.8 328.8 331.2
    lat_bnds   (lat, bnds) float32 200B 76.25 73.75 73.75 ... 16.25 16.25 13.75
    time_bnds  (time, bnds) object 47kB 2013-01-01 00:00:00 ... 2015-01-01 00...
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

Note that `ds.bounds.add_missing_bounds` uses `ds.bounds.add_bounds` for the latitude and longitude axes and defaults to the `frequency` method and `add_time_bounds` for the time axis. If you click on the database symbol for `time_bnds` above, the bounds are slightly mis-aligned because the time axis was not centered before adding the time axis. In this case, the user should call `xcdat.center_times` and then `ds.bounds.add_missing_bounds` (as shown earlier).


## Get the dimension coordinates for an axis.

In `xarray`, you can get a dimension coordinates by directly referencing its name (e.g., `ds.lat`). `xcdat` provides an alternative way to get dimension coordinates agnostically by simply passing the CF axis key to applicable APIs.

- Related API: [xcdat.get_dim_coords()](../generated/xcdat.get_dim_coords.rst) & [xcdat.get_dim_keys()](../generated/xcdat.get_dim_keys.rst)

Helpful knowledge:

- This API uses `cf_xarray` to interpret CF axis names and coordinate names in the xarray object attributes. Refer to [Metadata Interpretation](../getting-started-guide/faqs.rst) for more information.

Xarray documentation on coordinates ([source](https://docs.xarray.dev/en/stable/user-guide/data-structures.html#coordinates)):

- There are two types of coordinates in xarray:

  - **dimension coordinates** are one dimensional coordinates with a name equal to their sole dimension (marked by \* when printing a dataset or data array). They are used for label based indexing and alignment, like the index found on a pandas DataFrame or Series. Indeed, these “dimension” coordinates use a pandas.Index internally to store their values.

  - **non-dimension coordinates** are variables that contain coordinate data, but are not a dimension coordinate. They can be multidimensional (see Working with Multidimensional Coordinates), and there is no relationship between the name of a non-dimension coordinate and the name(s) of its dimension(s). Non-dimension coordinates can be useful for indexing or plotting; otherwise, xarray does not make any direct use of the values associated with them. They are not used for alignment or automatic indexing, nor are they required to match when doing arithmetic (see Coordinates).

- Xarray’s terminology differs from the [CF terminology](https://cfconventions.org/cf-conventions/v1.6.0/cf-conventions.html#terminology), where the “dimension coordinates” are called “coordinate variables”, and the “non-dimension coordinates” are called “auxiliary coordinate variables” (see [GH1295](https://github.com/pydata/xarray/issues/1295) for more details).


### 1. `axis` attr


In [21]:
ds.lat.attrs["axis"]

'Y'

### 2. `standard_name` attr


In [22]:
ds.lat.attrs["standard_name"]

'latitude'

In [23]:
"lat" in ds.dims

True

### Utilities to get the coordinate axis and coordinate axis key


In [24]:
xc.get_dim_coords(ds, axis="Y")

<xarray.DataArray 'lat' (lat: 25)> Size: 100B
array([75. , 72.5, 70. , 67.5, 65. , 62.5, 60. , 57.5, 55. , 52.5, 50. , 47.5,
       45. , 42.5, 40. , 37.5, 35. , 32.5, 30. , 27.5, 25. , 22.5, 20. , 17.5,
       15. ], dtype=float32)
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
Attributes:
    standard_name:  latitude
    long_name:      Latitude
    units:          degrees_north
    axis:           Y
    bounds:         lat_bnds

In [25]:
xc.get_dim_keys(ds, axis="X")

'lon'